# Analysis of specific junction

We are exemplary considering junction CIRMBUYJFK_f__CWCCKOQCWZ_r

In [1]:
# set working directory to project folder
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import mplcursors

import altair as alt
from itertools import combinations
import numpy as np
import pypangraph as pp
from Bio import Phylo, SeqIO, AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


from pathlib import Path
import subprocess

from Bio.Phylo.TreeConstruction import DistanceCalculator
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform

from junction_analyis.helpers import get_tree_order
import junction_analyis.pangraph_utils as pu
from junction_analyis.plotting import plot_junction_pangraph_combined, plot_junction_pangraph_interactive, plot_dendrogram
from junction_analyis.consensus import find_consensus_paths, make_deduplicated_paths
from junction_analyis.block_alignment import create_block_msas, summarize_block_msas, analyze_alignment, cluster_alignment, retrieve_cluster_members

## Redefine consensus paths based on sequence information
Analyze whether all consensus paths are unique or whether some should be merged.

In [ ]:
example_junction = "CIRMBUYJFK_f__CWCCKOQCWZ_r"
example_pangraph = pp.Pangraph.from_json(f"../results/junction_pangraphs/{example_junction}.json")

consensus_paths, path_dict, similarity_matrix, assignment_df = find_consensus_paths(example_pangraph, rare_block_threshold=5, rare_edge_threshold=5, min_n_isolates_per_consensus=5)

In [ ]:
block_id = consensus_paths[0].nodes[18].id
block_aln = example_pangraph.blocks[block_id].to_biopython_alignment()
AlignIO.write(block_aln, f"../results/marie_playground/{example_junction}_{block_id}_aln.fa", "fasta")

In [ ]:
block_id = 6932368721519041699
block_aln = example_pangraph.blocks[block_id].to_biopython_alignment()
AlignIO.write(block_aln, f"../results/marie_playground/{example_junction}_{block_id}_aln.fa", "fasta")

In [ ]:
block_2_id = consensus_paths[0].nodes[23].id
block_ids = [block_id, block_2_id, consensus_paths[0].nodes[6].id, consensus_paths[0].nodes[7].id, consensus_paths[0].nodes[26].id]

In [ ]:
summary_df = pd.read_csv(f"../results/block_alignments/{example_junction}/{example_junction}_alignment_stats.csv")

In [ ]:
summary_df[summary_df['block_id'].isin(block_ids)]

In [ ]:
summary_df[summary_df['block_id'] == consensus_paths[0].nodes[16].id]

In [ ]:
summary_df.sort_values('mismatch_fraction')

In [ ]:
summary_df.loc[39, 'block_id']

In [ ]:
example_pangraph.to_blockstats_df().loc[summary_df.loc[35, 'block_id']]

In [ ]:
example_pangraph.to_blockcount_df().loc[summary_df.loc[14, 'block_id']][example_pangraph.to_blockcount_df().loc[summary_df.loc[14, 'block_id']] == 1]

Findings for junction CIRMBUYJFK_f__CWCCKOQCWZ_r:
- Even though there are some mismatch columns in the alignment, those are usually point mutations found in one or maximally 2 sequences. These mutations could also stem from sequencing errors.
- Block 964531385716605116 (which is found in consensus path 1 only) has a lot of variation, however this stems from only one of the 13 isolates containing this blog
- for block 6932368721519041699 there are actually two different versions, it is a duplicated block, it is inserted at many different positions in the genome
- Core block 16494405553751752639 seems to have two versions, one version belongs to the short consensus path consensus_3, the other version belongs to the longer consensus path (1 and 2), the other core block 11400043001338627984 has the same two versions as well as all shared blocks
    - clustering isolates by core block similarity (e.g. first core block) shows perfect seperation between consensus_3 and consensus_1/2
    - wrongly forcing 3 clusters intermixes isolates between consensus_1 and consensus_2
    - we should have 2 consensus paths

In [ ]:
# TODO: find a better way to do this (cluster sequences, talk with Marco what would be best)
aln_path = f"../results/block_alignments/{example_junction}/block_11400043001338627984_aln.fa"
distance_matrix, Z, names = cluster_alignment(aln_path)

plot_dendrogram(Z, names)

In [ ]:
clusters = retrieve_cluster_members(Z, names, n_clusters=3)
plot_junction_pangraph_interactive(
    example_pangraph,
    show_consensus=True,
    consensus_paths=consensus_paths,
    assignments=assignment_df,
    order="tree",
    highlight_clusters=clusters
)

In [ ]:
clusters_2 = retrieve_cluster_members(Z, names, n_clusters=2)
plot_junction_pangraph_interactive(
    example_pangraph,
    show_consensus=True,
    consensus_paths=consensus_paths,
    assignments=assignment_df,
    order="tree",
    highlight_clusters=clusters_2
)

## Analyze deviations (insertions / deletions) from consensus path
Consider the two consensus paths seperately from each other.

In [ ]:
# choosing higher thresholds merges the two consensus paths into one, however this might not be possible for all junctions while still keeping all desired consensus paths

consensus_paths, path_dict, similarity_matrix, assignment_df = find_consensus_paths(example_pangraph, rare_block_threshold=10, rare_edge_threshold=10, min_n_isolates_per_consensus=5)
plot_junction_pangraph_interactive(
    example_pangraph,
    show_consensus=True,
    consensus_paths=consensus_paths,
    assignments=assignment_df,
    order="tree",
    highlight_clusters=clusters_2
)

### Consensus 2
Consensus path 2 only has two insertion sequences which I manually extracted.

In [ ]:
# as a starting point, start with one insertion sequence

# big insertion sequence in NZ_CP098219_1: 2278011600046483881
block_id = 2278011600046483881
isolate_name = "NZ_CP098219_1"

def write_block_fasta(example_pangraph, isolate_name, block_id, single_sequence = True):
    if single_sequence:
        sequence = Seq(example_pangraph.blocks[block_id].to_biopython_records()[0].seq)
    else:
        sequence = Seq(example_pangraph.blocks[block_id].consensus())
    record = SeqRecord(
        Seq(example_pangraph.blocks[block_id].to_biopython_records()[0].seq),
        id=f"{isolate_name}|block_{block_id}",
        description=f"block {block_id} from isolate {isolate_name}"
    )
    output_path = f"../results/atb_lookup/{isolate_name}_block_{block_id}.fasta"
    SeqIO.write(record, output_path, "fasta")

write_block_fasta(example_pangraph, isolate_name, block_id, single_sequence=True)

# results/marie_playground/NZ_CP098219_1_block_2278011600046483881.fasta look up in all the bacteria

In [ ]:
# read results from lexicmap run
atb_hits_df = pd.read_csv("../results/marie_playground/NZ_CP098219_1_block_2278011600046483881.lexicmap.tsv", sep="\t")
atb_hits_df.head()

In [ ]:
def write_sgenome_ids(atb_hits_df, output_file):
    sgenome_ids = atb_hits_df.sgenome.to_list()
    with open(output_file, "w") as f:
        for sid in sgenome_ids:
            f.write(str(sid) + "\n")

In [ ]:
# choose best matches and write biosample accession numbers into txt file to look them up in ncbi

atb_hits_df = atb_hits_df[atb_hits_df['qcovGnm'] > 99]
write_sgenome_ids(atb_hits_df, "../results/marie_playground/NZ_CP098219_1_block_2278011600046483881.ids.txt")

In [ ]:
# read results from ncbi

ncbi_results_df = pd.read_csv("../results/marie_playground/NZ_CP098219_1_block_2278011600046483881.ncbi_results.tsv", sep="\t")

In [ ]:
atb_hits_info_df = pd.merge(atb_hits_df, ncbi_results_df, on="sgenome", how="left")
atb_hits_info_df.to_csv("../results/marie_playground/NZ_CP098219_1_block_2278011600046483881.hits_info.tsv", index=False)

In [ ]:
atb_hits_info_df.organism.value_counts()

In [ ]:
# redo with another insertion sequence
write_block_fasta(example_pangraph, isolate_name = "NZ_AP022171.1", block_id = 18228278273347143766, single_sequence=True)

sbatch lexicmap_query.sh NZ_CP098219_1_block_2278011600046483881.fasta

In [ ]:
atb_hits_df = pd.read_csv("../results/atb_lookup/NZ_AP022171.1_block_18228278273347143766.lexicmap.tsv", sep="\t")
write_sgenome_ids(atb_hits_df, "../results/atb_lookup/NZ_AP022171.1_block_18228278273347143766.ids.txt")

bash fetch_biosample.sh NZ_AP022171.1_block_18228278273347143766.ids.txt NZ_AP022171.1_block_18228278273347143766.ncbi_results.tsv

In [ ]:
ncbi_results_df = pd.read_csv("../results/atb_lookup/NZ_AP022171.1_block_18228278273347143766.ncbi_results.tsv", sep="\t")
atb_hits_info_df = pd.merge(atb_hits_df, ncbi_results_df, on="sgenome", how="left")
atb_hits_info_df.to_csv("../results/atb_lookup/NZ_AP022171.1_block_18228278273347143766.hits_info.tsv", index=False)

In [ ]:
atb_hits_info_df.organism.value_counts()

Insertion sequence:
- toxin YdaT family protein, these are often small proteins associated with toxin–antitoxin (TA) systems, used by bacteria to regulate growth or defend against stress and phage infection.
- RISPR-Cas-Mediated Gene Silencing Reveals RacR To Be a Negative Regulator of YdaS and YdaT Toxins in Escherichia coli K-12

### Consensus 1
Derive all blocks with corresponding isolate name that deviate from the consensus path.

In [ ]:
# get a list of all blocks that deviate from consensus paths (consider insertions and deletions!)

# get isolates belonging to consensus 1
isolates_1 = assignment_df[assignment_df['best_consensus'] == 'consensus_1'].index.tolist()
isolates_1_set = set(isolates_1)

# make path dict
path_dict = example_pangraph.to_path_dictionary()
path_dict = {isolate: pu.Path.from_tuple_list(path, 'node') for isolate, path in path_dict.items() if isolate in isolates_1_set}

# deduplicate blocks
blockstats_df = example_pangraph.to_blockstats_df()
deduplicated_paths, deduplicated_blog_freq = make_deduplicated_paths(blockstats_df, path_dict)

# compare deduplicated paths to consensus paths to find deviations, consensus paths are already deduplicated


# insertions
# start with easier one
consensus_paths[0]



# potentially convert isolates to path data strucutes
example_pangraph.paths[isolates_1[0]]


In [ ]:
consensus_paths[0]

In [ ]:
path_dict